In [10]:
import pandas as pd
import xgboost as xgb
import numpy as np

In [11]:
df = pd.read_csv('../data/transformed_data.csv')

In [ ]:
df.head()

# Hyper paramétrisation

In [13]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [14]:
def display_train_test_metrics(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print(f"MAE  : {mean_absolute_error(y_test, y_pred)}")
    print(f"RMSE : {np.sqrt(mean_squared_error(y_test, y_pred))}")
    print(f"R²   : {r2_score(y_test, y_pred)}")

In [15]:
def search_hyperparameters(max_depths, min_child_weights, n_estimators, learning_rates, X_train, y_train, X_test, y_test):
    for i in max_depths:
        for j in min_child_weights:
            for k in n_estimators:
                for l in learning_rates:
                    print(f'max_depth: {i}, min_child_weight: {j}, n_estimators: {k}, learning_rate: {l}')
                    xgb_model = xgb.XGBRegressor(max_depth=i, min_child_weight=j, n_estimators=k, learning_rate=l, random_state=42)
                    display_train_test_metrics(xgb_model, X_train, y_train, X_test, y_test)

# Model prédiction Magnitude

In [16]:
from sklearn.model_selection import train_test_split
X = df.drop("mag", axis=1)
y = df["mag"]
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# search_hyperparameters([3, 6, 9], [1, 5, 10], [100, 500, 1000], [0.3, 0.15, 0.05], X_train_1, y_train_1, X_test_1, y_test_1)

In [18]:
X_train_1.shape

(84861, 41)

In [19]:
xgb_model_1 = xgb.XGBRegressor(random_state=42)

NameError: name 'xgb' is not defined

In [ ]:
xgb_model_1.fit(X_train_1, y_train_1)

## Analyse


In [ ]:
X_test_1.shape

In [ ]:
xgb.plot_importance(xgb_model_1)

In [ ]:


y_pred = xgb_model_1.predict(X_test_1)

print(f"MAE  : {mean_absolute_error(y_test_1, y_pred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test_1, y_pred))}")
print(f"R²   : {r2_score(y_test_1, y_pred)}")

# Model prédiction Depth

In [ ]:
from sklearn.model_selection import train_test_split
X = df.drop("depth", axis=1)
y = df["depth"]
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# search_hyperparameters([3, 6, 9], [1, 5, 10], [100, 500, 1000], [0.3, 0.15, 0.05], X_train_1, y_train_1, X_test_1, y_test_1)

In [ ]:
X_train_2.shape

In [ ]:
xgb_model_2 = xgb.XGBRegressor(random_state=42)

In [ ]:
xgb_model_2.fit(X_train_2, y_train_2)

## Analyse

In [ ]:
X_test_1.shape

In [ ]:
xgb.plot_importance(xgb_model_1)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_pred = xgb_model_2.predict(X_test_2)

print(f"MAE  : {mean_absolute_error(y_test_2, y_pred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test_2, y_pred))}")
print(f"R²   : {r2_score(y_test_2, y_pred)}")